# Actividad 11: ANOVA de dos vías

En esta actividad se presenta un **ANOVA de dos vías** usando como variables de ejemplo:

- Respuesta: `ta_micromol_kg`  
- Factores: `season` y `estuary`

**Nota importante:**  
Sabemos que los datos **no cumplen con el supuesto de normalidad**, por lo que este análisis **no sería válido en un contexto real**. Sin embargo, incluimos este ejercicio en el repositorio porque:

1. Queremos **completar la secuencia de actividades** del curso.  
2. Sirve como **ejemplo de referencia y práctica** para entender la estructura de un ANOVA de dos vías en Python.  
3. Nos será útil como **consulta futura** para aplicar con datasets adecuados que sí cumplan los supuestos.  

De esta forma, aunque aquí los resultados no deben interpretarse como conclusiones estadísticas reales, sí nos familiarizamos con la sintaxis y flujo de trabajo:  
- Ajuste del modelo con `statsmodels`  
- Tabla ANOVA  
- Pruebas post-hoc (Tukey HSD) en caso de efectos significativos.


In [3]:
import pandas as pd

# Cargar datos
def import_csv(file):
    return pd.read_csv(file)

path = "../data/Terminos_lagoon_TA_DIC_2023_RawData.csv"
co2_data = import_csv(path)

co2_data.head()


,sample,date,estuary,area,station,layer_depth,season,chlorophy_microg_l,cond_microsiemens_cm,depth_m,...,do_mg_l,sal_psu,sp_cond_microsiemens_cm,turbidity_fnu,temp_c,latitude,longitude,dic_micromol_kg,ta_micromol_kg,dummy_data
0,CDL01S,5/3/2020,Candelaria,River,CDL01,Surface,Dry,0.36,7015.4,0.464,...,7.12,3.56,6547.7,1.47,28.740,18.55736,-91.25012,3915,3863,3685.0
1,CDL01F,5/3/2020,Candelaria,River,CDL01,Bottom,Dry,4.19,29886.1,7.792,...,4.90,16.97,27751.2,95.33,29.028,18.55722,-91.24990,3698,3685,NaN
2,CDL02S,5/3/2020,Candelaria,River,CDL02,Surface,Dry,0.92,16691.1,0.453,...,6.99,8.94,15429.1,5.50,29.283,18.61007,-91.24410,3724,3708,3708.0
3,CDL02F,5/3/2020,Candelaria,River,CDL02,Bottom,Dry,2.23,24847.4,1.261,...,6.52,13.87,23074.0,13.44,29.024,18.61005,-91.24403,3667,3992,3992.0
4,CDL03S,5/3/2020,Candelaria,River,CDL03,Surface,Dry,0.58,46341.6,0.465,...,6.24,28.06,43670.8,3.60,28.202,18.63166,-91.29359,2928,3023,3023.0


In [4]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def two_way_anova_tukey(df, response, factor1, factor2, alpha=0.05):
    """
    Corre ANOVA de dos vías con interacción y, si hay efectos
    significativos, aplica Tukey HSD como post-hoc.
    """
    # Trabajar con copia limpia y eliminar filas con nulos en las columnas relevantes
    data = df[[response, factor1, factor2]].dropna().copy()

    # Convierte factores a categóricos por seguridad
    data[factor1] = data[factor1].astype("category")
    data[factor2] = data[factor2].astype("category")

    # Info de tamaños por grupo (útil para revisar desbalanceos)
    group_sizes = data.groupby([factor1, factor2])[response].size().unstack(fill_value=0)
    print("Tamaños por grupo (conteos):")
    display(group_sizes)

    # Ajuste del modelo con interacción
    formula = f'{response} ~ C({factor1}) + C({factor2}) + C({factor1}):C({factor2})'
    model = ols(formula, data=data).fit()

    # Tabla ANOVA (Tipo II)
    anova_table = sm.stats.anova_lm(model, typ=2)
    print("\nResultados ANOVA (Tipo II):")
    display(anova_table)

    # Determinar si hay algún efecto significativo
    any_sig = (anova_table["PR(>F)"] < alpha).any()

    if any_sig:
        print("\nSe detectaron efectos significativos (p < α). Ejecutando Tukey HSD por combinaciones de factores...")
        # Crear un grupo combinado para Tukey (todas las celdas factor1-factor2)
        data["_Group"] = data[factor1].astype(str) + " - " + data[factor2].astype(str)
        tukey = pairwise_tukeyhsd(data[response], data["_Group"], alpha=alpha)
        print(tukey.summary())
    else:
        print("\nNo se detectaron efectos significativos (p ≥ α). No se requiere post-hoc.")

    return anova_table


## Ejemplo del código del curso

In [5]:
_ = two_way_anova_tukey(
    df=co2_data,
    response="ta_micromol_kg",
    factor1="season",
    factor2="estuary",
    alpha=0.05
)


Tamaños por grupo (conteos):


/var/folders/1q/0d1wc4291_z5gfjdgg375t4m0000gn/T/ipykernel_15951/34647470.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_sizes = data.groupby([factor1, factor2])[response].size().unstack(fill_value=0)


estuary,Candelaria,Palizada
season,,
Dry,36,36
Rainy,16,18



Resultados ANOVA (Tipo II):


,sum_sq,df,F,PR(>F)
C(season),7.234655e+06,1.0,45.502782,9.393830e-10
C(estuary),9.238908e+03,1.0,0.058109,8.099945e-01
C(season):C(estuary),8.689347e+03,1.0,0.054652,8.156259e-01
Residual,1.621735e+07,102.0,NaN,NaN



Se detectaron efectos significativos (p < α). Ejecutando Tukey HSD por combinaciones de factores...
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
      group1             group2        meandiff p-adj    lower     upper   reject
---------------------------------------------------------------------------------
  Dry - Candelaria     Dry - Palizada   31.1111 0.9874 -214.3649  276.5871  False
  Dry - Candelaria Rainy - Candelaria -539.7153 0.0001  -852.637 -226.7935   True
  Dry - Candelaria   Rainy - Palizada -547.4444    0.0   -848.09 -246.7989   True
    Dry - Palizada Rainy - Candelaria -570.8264    0.0 -883.7482 -257.9046   True
    Dry - Palizada   Rainy - Palizada -578.5556    0.0 -879.2011   -277.91   True
Rainy - Candelaria   Rainy - Palizada   -7.7292 0.9999 -365.5689  350.1106  False
---------------------------------------------------------------------------------


## Ejercicio 2: DIC

In [6]:
_ = two_way_anova_tukey(
    df=co2_data,
    response="dic_micromol_kg",
    factor1="season",
    factor2="estuary",
    alpha=0.05
)


Tamaños por grupo (conteos):


/var/folders/1q/0d1wc4291_z5gfjdgg375t4m0000gn/T/ipykernel_15951/34647470.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  group_sizes = data.groupby([factor1, factor2])[response].size().unstack(fill_value=0)


estuary,Candelaria,Palizada
season,,
Dry,36,36
Rainy,16,18



Resultados ANOVA (Tipo II):


,sum_sq,df,F,PR(>F)
C(season),4.720620e+06,1.0,22.414493,0.000007
C(estuary),5.452007e+04,1.0,0.258873,0.611995
C(season):C(estuary),1.863361e+03,1.0,0.008848,0.925245
Residual,2.148178e+07,102.0,NaN,NaN



Se detectaron efectos significativos (p < α). Ejecutando Tukey HSD por combinaciones de factores...
               Multiple Comparison of Means - Tukey HSD, FWER=0.05               
      group1             group2        meandiff p-adj    lower     upper   reject
---------------------------------------------------------------------------------
  Dry - Candelaria     Dry - Palizada   51.1389 0.9649 -231.3844  333.6622  False
  Dry - Candelaria Rainy - Candelaria -442.9306 0.0094 -803.0785  -82.7826   True
  Dry - Candelaria   Rainy - Palizada -409.7778 0.0134 -755.7967  -63.7588   True
    Dry - Palizada Rainy - Candelaria -494.0694 0.0029 -854.2174 -133.9215   True
    Dry - Palizada   Rainy - Palizada -460.9167 0.0041 -806.9356 -114.8977   True
Rainy - Candelaria   Rainy - Palizada   33.1528 0.9967 -378.6921  444.9977  False
---------------------------------------------------------------------------------


## Nota de cierre

Este notebook queda como **plantilla** para futuros análisis con datasets que sí cumplan supuestos (normalidad y homogeneidad de varianzas).  
Pasos a replicar:
1. Seleccionar la variable respuesta y factores.  
2. Eliminar nulos y convertir factores a `category`.  
3. Ajustar modelo con interacción
4. Revisar la **tabla ANOVA** y, si hay efectos significativos, aplicar **Tukey HSD**.  
